In [ ]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [ ]:
#calculate flux coming from one defined source
#detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'na', 'nb'])
detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5'])
echan = 2
#detectors = np.array(['n3', 'n5', 'n0', 'n9'])

day = 140611
burst_ra = 110.337
burst_dec = 11.484
binning = 11

start_time = 4.8
end_time = 5.0


burst_occ = calc.src_occultation(day, burst_ra, burst_dec)[0]

#sat_time = rf.poshist(day)[0]

#plot_time_sat_date = calc.met_to_date(sat_time)[0]
#plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours

mpl.rcParams['axes.color_cycle'] = ['g', 'r', 'c', 'm', 'y', 'k']

tot_residuals = []
tot_counts = []

for i in range(0, len(detectors)):
    #print detectors[i]
    ang_data = calc.burst_ang_bin(detectors[i], day, burst_ra, burst_dec)
    det_ang = ang_data[0]
    
    det_ang_eff = calc.ang_eff(det_ang, echan)[0]
    #sat_time = det_data[1]
    
    ctime_data = calc.curve_fit_plots(day, detectors[i], echan)
    det_residuals = ctime_data[0]
    det_counts = ctime_data[1]
    det_fit = ctime_data[2]
    plot_time_bin = ctime_data[10]
    plot_time_sat = ctime_data[11]
    
    eff_residuals = det_residuals*det_ang_eff
    eff_counts = det_counts*det_ang_eff
    
    binned_res = eff_residuals[0::binning]
    binned_counts = eff_counts[0::binning]
    binned_times_bin = plot_time_bin[int(binning/2)::binning]
    binned_times_sat = plot_time_sat[int(binning/2)::binning]
    
    for j in range(1, binning):
        if len(binned_res) != len(eff_residuals[j::binning]):
            binned_res = np.delete(binned_res, -1)
            binned_counts = np.delete(binned_counts, -1)
        binned_res = np.add(binned_res, eff_residuals[j::binning])
        binned_counts = np.add(binned_counts, eff_counts[j::binning])
    
    if len(binned_res) < len(binned_times_bin):
        binned_times_bin = np.delete(binned_times_bin, -1)
        binned_times_sat = np.delete(binned_times_sat, -1)
    
    tot_residuals.append(binned_res)
    tot_counts.append(binned_counts)
    
    #det_ang[np.where(det_ang > 90.)] = 0.
    
    plt.plot(binned_times_bin, binned_res, label = detectors[i])
    
    
    
tot_residuals = np.array(tot_residuals)
tot_counts = np.array(tot_counts)
tot_residuals = np.squeeze(np.sum(tot_residuals, axis = 0))
tot_counts = np.squeeze(np.sum(tot_counts, axis = 0))
    
#print plot_time_bin
#print tot_residuals
    
plt.plot(binned_times_bin, tot_residuals, 'b-', label = 'total residuals')
plt.legend()
    
plt.show()